# Actividad grupal: Usos reales de filtros espaciales y morfológicos

## Introduccion

Los filtros morfológicos tienen como finalidad manipular la composición de las imágenes para poder describir su composición geométrica. 

Entre las operaciones morfológicas existen dos principales: dilatación y erosión, y sus combinaciones binarias: apertura y cierre.

La dilatación suele generar un efecto de ampliación del objeto en una imágen, llegando a eliminar algunos detalles que se consideren mínimos o ajenos al objeto que conforma la mayor parte de la composición de la misma. La erosión genera un efecto contrario, el cual amplia los detalles mínimos encontrados ajenos al agente principal de la imagen, minimizando el objeto que tenga mayor presencia en la misma.

Ambas operaciones se pueden ciclar e iterar en la forma que convenga para obtener un efecto deseado en determinado tipo de imagenes.

En el presente caso utilizamos imagenes de rayos X, que por la naturaleza de las mismas presentan alto contraste y escalas de grises. Estas imagenes corresponden directamente a consulta médica donde existió un agente extraño en el cuerpo humano y con las operaciones de filtros morfológicos pretendemos hacer más sencilla su determinación de posible diagnóstico y tratamiento médico. La imágeness presentaron resultados favorables para las operaciones en cuestión.


## Librerias a utilizar

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
def padding(originalImg, padSize):
    padImg = np.zeros((rows+2*padSize, columns+2*padSize), dtype=np.uint8)
    # recortando
    padImg[padSize:rows+padSize, padSize:columns+padSize] = originalImg
    return padImg

In [ ]:
def Erosion(padImg, kernel, size):
    output = np.zeros((rows, columns), dtype=np.uint8)
    for i in range(0, rows):
        for j in range(0, columns):
            # recortando
            portion = padImg[i:i+size, j:j+size]
            # se suma el elemento estructural y la ventana
            portion1 = portion.flatten()
            portion2 = kernel.flatten()
            p1 = (np.sum(portion1))
            p2 = (np.sum(portion2))*255
            # la condicional para que no revase el limite
            if p1 == p2:
                output[i, j] = 255
            else:
                output[i, j] = np.min(portion1)
    return output

In [ ]:
def Dilatacion(padImg, size):
    output = np.zeros((rows, columns), dtype=np.uint8)
    for i in range(0, rows):
        for j in range(0, columns):
            # recortando
            portion = padImg[i:i+size, j:j+size]
            portion1 = portion.flatten()
            # la condicional para que no revase el limite
            if 255 in portion1:
                output[i, j] = 255
            else:
                output[i, j] = np.max(portion1)
    return output

In [ ]:
def opening(padImg, kernel, size):
    # Se aplica la erosion
    erosion = Erosion(padImg, kernel , size)
    padImg2 = padding(erosion, size//2)
    # Se aplica al dilatacion
    output = Dilatacion(padImg2, size)
    return output

In [ ]:
def closing(padImg,kernel, size):
    # Se aplica al dilatacion
    dilation = Dilatacion(padImg, size)
    padImg2 = padding(dilation, size//2)
    # Se aplica la erosion
    output = Erosion(padImg2, kernel, size)
    return output

In [ ]:

#Numero de elementos de la mascara a utilizar
size = 9

#Elemento estructural
kernel = np.ones((size, size), np.uint8)
print(np.sum(kernel))

#Tamaño del padding
p_size = size//2

#Lectura de la imagen a utilizar
orginalImg = cv.imread('dental 10.JFIF', 0)

#Se obtiene los tamaños de la imagen original
rows = orginalImg.shape[0]
columns = orginalImg.shape[1]

#Se obtiene el padding inicial
padImg = padding(orginalImg, p_size)

#Aplicamos dilatacion
Dil = Dilatacion(padImg, size)

#Aplicamos erosion
Ero = Erosion(padImg, kernel, size)

#Gradiante Morfologico
Gradiante = Dil - Ero
cv.imshow('Gradiante Morfologico', Gradiante)
cv.waitKey(0)

## Conclusiones

## Bibliografia 

https://medium.com/swlh/image-processing-with-python-morphological-operations-26b7006c0359

González, R. y R.E. Woods, 2006. Digital Image Processing. New Jersey, Prentice Hall, 
(3ª edición).

Ruiz Fernández, L.A, 2020, Universitat Politècnica de València, Ingeniería Cartográfica, Geodesia y 
Fotogrametría.

http://www.ece.northwestern.edu/local-apps/matlabhelp/toolbox/images/morph3.html